<a href="https://colab.research.google.com/github/Vinaypatil-Ev/vinEvPy-GoCoLab/blob/main/Tensorflow/TesorflowPrac17_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

##1. adapt() method

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
data = ["thiss is first line", "This is second line", "This is third line", "This is fourth line", "This is fifth line"]
layer = preprocessing.TextVectorization()
layer.adapt(data)

In [ ]:
layer(data)

<tf.Tensor: shape=(5, 4), dtype=int64, numpy=
array([[ 5,  3,  9,  2],
       [ 4,  3,  7,  2],
       [ 4,  3,  6,  2],
       [ 4,  3,  8,  2],
       [ 4,  3, 10,  2]])>

## Image data Augmentation

In [ ]:
augment_layer = tf.keras.Sequential([preprocessing.RandomFlip("horizontal"),
                                     preprocessing.RandomRotation(0.1),
                                     preprocessing.RandomZoom(0.1),
])

In [ ]:
input_shape = (32, 32, 3)
classes = 10


In [ ]:
inputs = tf.keras.Input(
    shape=input_shape)
x = augment_layer(inputs)
x = preprocessing.Rescaling(1.0 / 255)(x)
outputs = tf.keras.applications.ResNet50(weights=None, input_shape=input_shape, classes=classes)(x)
model = tf.keras.Model(inputs, outputs)

## Noramalization

In [ ]:
(xtrn, ytrn), (_, _) = tf.keras.datasets.cifar10.load_data()
data = xtrn.reshape((len(xtrn), -1))
input_shape = data.shape[1:]
classes=10

In [ ]:
normalize_layer = preprocessing.Normalization()
normalize_layer.adapt(data)

In [ ]:
inputs = tf.keras.Input(shape=input_shape)
x = normalize_layer(inputs)
outputs = tf.keras.layers.Dense(classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy")
model.fit(data, ytrn)

1563/1563 [==============================] - 4s 2ms/step - loss: 2.1959


In [ ]:
data = tf.constant(["a", "b", "c", "d", "", "  "," "," "])

In [ ]:
x = preprocessing.StringLookup()
x.adapt(data)
x(data)

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([6, 5, 4, 3, 0, 7, 2, 2])>

In [ ]:
x1 = preprocessing.CategoryEncoding(output_mode="binary")
x1.adapt(x(data))
# data = tf.constant(["a", "b", "c", "d", ""])
x1(x(data))

<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

In [ ]:
x(data)

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([5, 4, 3, 2])>

## Hashing (for large no of categories in onehot range:1e3)

In [ ]:
import numpy as np


In [ ]:
data = np.random.randint(0, 10000, size=(10000, 1))
hash = preprocessing.Hashing(num_bins=64, salt=1337)
o = preprocessing.CategoryEncoding(max_tokens=64, output_mode="binary")
o = o(hash(data))
o.shape

TensorShape([10000, 64])

In [ ]:
def encode_text_model(data, mode="tf-idf", ngrams=None):
    tvector = preprocessing.TextVectorization(output_mode=mode, ngrams=ngrams)
    tvector.adapt(data)

    inputs = tf.keras.Input(shape=(1,), dtype="string")
    x = tvector(inputs)
    outputs = tf.keras.layers.Dense(10)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)

test_data = tf.constant(["The Brain is deeper than the see"])

In [ ]:
encoder = ["tf-idf", "binary"]
for i in encoder:
    model = encode_text_model(data, i, 2)
    print(model(test_data).numpy().argmax())
    print("_______________")

6
_______________
7
_______________


In [ ]:
36*13.50

486.0

In [ ]:
np.array([10, 2, 3, 4, 5]).argmax()

0

In [ ]:
tvector = preprocessing.TextVectorization()
tvector.adapt(data)
inputs = tf.keras.Input(shape=(1, ), dtype="string")
x = tvector(inputs)
vocab = tvector.get_vocabulary()
x = tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=64)(x)
outputs = tf.keras.layers.LSTM(1)(x)
model = tf.keras.Model(inputs, outputs)
model(test_data)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00553942]], dtype=float32)>